## RF linac example



In [1]:
from abel import *
import numpy as np
%load_ext autoreload
%autoreload 2

## Define the linac and beams

In [2]:
# define beam
driver_source = SourceBasic()
driver_source.charge = -8e-9 # [C] bunch charge (not train)
driver_source.energy = 0.1e9 # [eV]
driver_source.rel_energy_spread = 0.01
driver_source.bunch_length = 300e-6 # [m]
driver_source.emit_nx, driver_source.emit_ny = 20e-6, 80e-6 # [m rad]
driver_source.beta_x = 1
driver_source.beta_y = driver_source.beta_x
driver_source.num_particles = 5000
driver_source.num_bunches_in_train = 3200
driver_source.bunch_separation = 4e-9
driver_source.rep_rate_trains = 100

driver_rf_accelerator = RFAcceleratorBasic()
driver_rf_accelerator.rf_frequency = 1e9 # L-band
driver_rf_accelerator.nom_energy_gain = 7.5e9
driver_rf_accelerator.nom_accel_gradient = 3e6
driver_rf_accelerator.rep_rate_trains = driver_source.rep_rate_trains
driver_rf_accelerator.operating_temperature = 300
driver = driver_rf_accelerator.track(driver_source.track())

print(f'Klystron peak power: {driver_rf_accelerator.get_klystron_peak_power()/1e6:.1f} MW')
print(f'Length: {driver_rf_accelerator.length/1e3:.1f} km')

print(f'Wall plug power: {driver_rf_accelerator.wallplug_power()/1e6:.1f} MW')
print(f'       RF power: {driver_rf_accelerator.wallplug_power_rf()/1e6:.1f} MW')
print(f'  Cooling power: {driver_rf_accelerator.wallplug_power_cooling()/1e6:.1f} MW')
driver_rf_accelerator.print_cost()

Klystron peak power: 12.9 MW
Length: 2.5 km
Wall plug power: 31.9 MW
       RF power: 31.9 MW
  Cooling power: 0.0 MW
-- COSTS -----------------------------------------
-- Instrumented beamline (29%): 29 MILCU
-- RF structures (1184x): 204 MILCU
-- Klystrons (1185x, 13 MW peak, 27 kW avg): 661 MILCU
--------------------------------------------------
-- Total: 0.89 BILCU
--------------------------------------------------


In [17]:
from abel.classes.combiner_ring.impl.combiner_ring_basic import CombinerRingBasic

cr = CombinerRingBasic()
cr.compression_factor = 5

comp_driver = cr.track(driver)
print(cr.get_length())
print(driver.train_duration(), '->', comp_driver.train_duration())

767.4686924800001
1.2796e-05 -> 2.5592e-06


In [ ]:
psource = SourceBasic()
psource.charge = 1e-9 # [C] bunch charge (not train)
psource.energy = 3e9 # [eV]
psource.rel_energy_spread = 0.01
psource.bunch_length = 100e-6 # [m]
psource.emit_nx, psource.emit_ny = 10e-6, 0.035e-6 # [m rad]
psource.beta_x = 1
psource.beta_y = psource.beta_x
psource.num_particles = 5000
psource.num_bunches_in_train = 133
psource.bunch_separation = 5.26e-9
psource.rep_rate_trains = 120

#p_rf_accelerator = RFAcceleratorCLICopti()
p_rf_accelerator = RFAcceleratorBasic()
p_rf_accelerator.rep_rate_trains = psource.rep_rate_trains
p_rf_accelerator.nom_energy_gain = 122e9
p_rf_accelerator.rf_frequency = 5.712e9 # C-band
p_rf_accelerator.operating_temperature = 77
p_rf_accelerator.fill_factor = 0.9
p_rf_accelerator.num_rf_cells = 1
p_rf_accelerator.num_structures_per_klystron = 130
p_rf_accelerator.nom_accel_gradient = 63e6
p_rf_accelerator.track(psource.track())

print(f'Beam power: {psource.charge * p_rf_accelerator.nom_energy_gain * psource.num_bunches_in_train * psource.rep_rate_trains/1e6:.1f} MW')
print(f'Length: {p_rf_accelerator.length/1e3:.1f} km')

print(f'Wall plug power: {p_rf_accelerator.wallplug_power()/1e6:.1f} MW')
print(f'       RF power: {p_rf_accelerator.wallplug_power_rf()/1e6:.1f} MW')
print(f'  Cooling power: {p_rf_accelerator.wallplug_power_cooling()/1e6:.1f} MW')
p_rf_accelerator.print_cost()

In [ ]:
#Geometry only
print(f"Linac fill factor = {rf_accelerator.fill_factor}")
print(f"Structure filling time = {rf_accelerator.get_fill_time()*1e9:.1f} [ns]")

In [ ]:
#Basic gradient/voltage information
#rf_accelerator.gradient_structure = 100e6 #It will figure out the voltages from the geometry
#rf_accelerator.track(source.track())
print(f"Total voltage = {rf_accelerator.nom_energy_gain/1e9:.1f} [GV], per structure = {rf_accelerator.voltage_structure/1e6:.1f} [MV]")

In [ ]:
print("Power per structure =", rf_accelerator.get_structure_power()/1e6, "[MW]")
print("Beam Pulse Length =", rf_accelerator.train_duration, "[ns]")
print("Total RF->beam Efficiency =", rf_accelerator.get_rf_efficiency()*100, "[%] (single bunch)")
print("Energy per structure =", rf_accelerator.get_structure_pulse_energy(), "[J]")
print("Max pulse length =", rf_accelerator.get_pulse_length_max()*1e9, "[ns] (beam current undefined)")
print("Max pulse length =", rf_accelerator.get_pulse_length_max()*1e9, "[ns] @ beam_current=",rf_accelerator.average_current_train, "[A]")

## Let's do some tracking - this changes the beam settings in the driverLinac object!

In [ ]:
beam.plot_lps()

print("Beam parameters in linac before tracking:",)
print(driverLinac)

beam2 = driverLinac.track(beam)
beam2.plot_lps()

print("Beam parameters in linac after tracking:",)
print(driverLinac)


In [ ]:
driverLinac.plot_gradient_profile()
plt.grid()

In [ ]:
driverLinac.plot_power_profile()
plt.grid()


## Gradient optimalization

We have now set the pulse length (from the train) and the total energy we want. What are the gradient we can achieve, and how long does the linac need to be?

In [ ]:
print(f"Max gradient at I_beam={driverLinac.average_current_train:.1f}[A], t_beam={driverLinac.train_duration*1e9:.1f}[ns]")
print(driverLinac)

#driverLinac._RF_structure.uselimit_PC = False

print("Current structure voltage  =", driverLinac.voltage_structure/1e6, "[MV] ->", driverLinac.gradient_structure/1e6, ", [MV/m] -> power/structure=", driverLinac.get_structure_power()/1e6, "[MW]")
print("Max structure voltage      =", driverLinac.get_structure_voltage_max()/1e6, "[MV]") #Max gradient at current pulse length and current
print("Max structure gradient     =", driverLinac.get_structure_voltage_max()/1e6/driverLinac.structure_length, "[MV/m]")
print("Max pulse length =", driverLinac.get_pulse_length_max()*1e9, "[ns]") #Max pulse length at current gradient and current
print( driverLinac._RF_structure.getMaxAllowableBeamTime_detailed(driverLinac.get_structure_power(), driverLinac.average_current_train))

In [ ]:

print("Fixing gradient and linac geometry")
print("optimizing...:", driverLinac.optimize_linac_geometry_and_gradient())
print(driverLinac)
print("Current structure voltage  =", driverLinac.voltage_structure/1e6, "[MV] ->", driverLinac.gradient_structure/1e6, ", [MV/m] -> power/structure=", driverLinac.get_structure_power()/1e6, "[MW]")
print("Max structure voltage      =", driverLinac.get_structure_voltage_max()/1e6, "[MV/m]") #Max gradient at current pulse length and current
print("Max structure gradient     =", driverLinac.get_structure_voltage_max()/1e6/driverLinac.structure_length, "[MV/m]")
print("Max pulse length           =", driverLinac.get_pulse_length_max()*1e9, "[ns]") #Max pulse length at current gradient and current
print( driverLinac._RF_structure.getMaxAllowableBeamTime_detailed(driverLinac.get_structure_power(), driverLinac.average_current_train)) 

# Exercising the RFAcceleratorBasic a bit

In [ ]:
basic = RFAcceleratorBasic(nom_energy_gain=1e9)
basic.num_structures = 100
basic.length=basic.num_structures*basic.structure_length/basic.autoOptimize_targetFillFactor
basic.track(beam,verbose=1)


# Basic test of SCRFAcceleratorBasic

In [ ]:
scrf = SCRFAcceleratorBasic(nom_energy_gain=500e6,rf_frequency=2e9, rep_rate_trains=10.0)
scrf.num_structures = 100
scrf.length=basic.num_structures*basic.structure_length/basic.autoOptimize_targetFillFactor
scrf.track(beam,verbose=1)